# load data

In [147]:
import numpy as np
import pandas as pd
import math
import json

In [78]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/pharmaHacks
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1coU_UapTjwPnfVvEmkKmrIYaovVuc4lv/pharmaHacks
/content/drive/.shortcut-targets-by-id/1coU_UapTjwPnfVvEmkKmrIYaovVuc4lv/pharmaHacks


In [79]:
!ls

 drugbank_vocabulary.csv	     'Similar drug finder_malinda.ipynb'
 old				     'Similar drug finder_molly.ipynb'
'Similar drug finder_anthony.ipynb'   sm_trails.csv
'Similar drug finder.ipynb'	     'STEPS-clinical trail app.gdoc'


## clinical trials data cleaning

In [5]:
df_ct = pd.read_csv('/content/drive/MyDrive/pharmaHacks/sm_trails.csv', index_col=0)
df_ct

,NCT Number,Title,Acronym,Status,Study Results,Conditions,Interventions,Outcome Measures,Sponsor/Collaborators,Gender,...,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,URL
Rank,,,,,,,,,,,,,,,,,,,,,
1,NCT00579683,Registry for Patients With Acquired Resistance...,NaN,Completed,No Results Available,Unresectable or Metastatic Non-small Cell Lung...,Other: Tumor core biopsy for RNA isolation,To compare EGFR gene sequence in patients upon...,Memorial Sloan Kettering Cancer Center,All,...,04-103,August 2004,February 2016,February 2016,"December 24, 2007",NaN,"February 4, 2016","Memoral Sloan Kettering Cancer Center, Basking...",NaN,https://ClinicalTrials.gov/show/NCT00579683
2,NCT00671671,Phase 1 Study To Evaluate Antiviral Activity O...,NaN,Completed,Has Results,"Hepatitis, Chronic|Hepatitis C Virus",Drug: Small Molecule Agent (PF-868554),Change From Baseline in Plasma Log10 Hepatitis...,Pfizer,All,...,A8121006,April 2008,December 2008,December 2008,"May 5, 2008","January 14, 2014","January 14, 2014","Pfizer Investigational Site, Gainesville, Flor...",NaN,https://ClinicalTrials.gov/show/NCT00671671
3,NCT02179671,Immune-Modulated Study of Selected Small Molec...,NaN,Completed,Has Results,Locally Advanced or Metastatic Non-Small-Cell ...,Drug: Gefitinib|Drug: AZD9291|Drug: Selumetini...,Confirmed Complete Response (CR) Rate|Objectiv...,"AstraZeneca|Quintiles, Inc.",All,...,D4191C00011,"July 25, 2014","June 11, 2016","June 11, 2016","July 2, 2014","August 2, 2019","August 2, 2019","Research Site, Goodyear, Arizona, United State...",NaN,https://ClinicalTrials.gov/show/NCT02179671
4,NCT02259114,"A Dose-Finding Study of Birabresib (MK-8628), ...",NaN,Completed,Has Results,NUT Midline Carcinoma|Triple Negative Breast C...,Drug: Birabresib,Number of Participants Who Experienced a Dose ...,Oncoethix GmbH,All,...,8628-003|OTX015_108|2014-002680-15|MK-8628-003,"October 23, 2014","March 3, 2017","March 3, 2017","October 8, 2014","October 1, 2018","January 26, 2021",NaN,"""Study Protocol"", https://ClinicalTrials.gov/P...",https://ClinicalTrials.gov/show/NCT02259114
5,NCT02395601,A Study Evaluating CPI-1205 in Patients With B...,NaN,Completed,No Results Available,B-Cell Lymphoma,Drug: CPI-1205,Frequency of Dose-limiting toxicities (DLTs)|F...,Constellation Pharmaceuticals,All,...,1205-01,March 2015,December 2018,December 2018,"March 23, 2015",NaN,"September 6, 2019","Horizon Oncology Center, Lafayette, Indiana, U...",NaN,https://ClinicalTrials.gov/show/NCT02395601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,NCT01306149,Natural History of Mineral Metabolism Paramete...,NaN,Completed,No Results Available,Complication of Peritoneal Dialysis,NaN,natural history of clearances and serum parame...,Universitaire Ziekenhuizen Leuven,All,...,FWO01-03-21,July 2002,"January 31, 2017","January 31, 2017","March 1, 2011",NaN,"October 25, 2017","University Hospital, Leuven, Vlaams-Brabant, B...",NaN,https://ClinicalTrials.gov/show/NCT01306149
284,NCT00935792,Everolimus and Alemtuzumab in Treating Patient...,NaN,Completed,Has Results,Lymphocytic Leukemia,Drug: alemtuzumab|Drug: everolimus,Clinical Response (Complete or Partial Remissi...,Mayo Clinic,All,...,MC088C|NCI-2009-00935|08-008775,July 2009,December 2015,December 2015,"July 9, 2009","December 12, 2016","September 26, 2017","Mayo Clinic in Arizona, Scottsdale, Arizona, U...",NaN,https://ClinicalTrials.gov/show/NCT00935792
285,NCT01456767,Effects of Administration of L.Plantarum WCFS1...,NaN,Completed,No Results Available,Gastro Intestinal Infection,Dietary Supplement: Probiotics,"Change in sugar recovery in urine, as indicato...",Maastricht University Medical Center,All,...,NL35728.068.11/METC11-2-011,October 2011,April 2012,NaN,"October 21, 2011",NaN,"July 19, 2012","Health, Medicine and Life Sciences, Maastricht...",NaN,https://ClinicalTrials.gov/show/NCT01456767


In [121]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

dict_ct = {}
for ii in range(df_ct.shape[0]):
    nct_no = df_ct.iloc[ii,0] # 'NCT Number'
    x = df_ct.iloc[ii,6] # 'Interventions'
    extracted = []
    # print(x)
    if isinstance(x, str):
        temp1 = list(find_all(x, 'Drug: '))
        temp2 = list(find_all(x, '|'))
        for i1 in temp1:
            index1 = i1+6
            index2 = -1
            for i2 in temp2:
                if i2 >= index1:
                    index2 = i2
                    break
            if index2 == -1:
                index2 = len(x)
            temp_str = x[index1:index2]
            temp_str = temp_str.replace('(','').replace(')','').replace('+',' ').replace('/',' ')
            temp_str1 = temp_str.split()
            temp_str2 = []
            for s in temp_str1:
                if not (len(s)==1 or s=='and' or s.endswith('mg') or s=='for' or s=='of' or s=='Placebo' or s.isnumeric()):
                    temp_str2.append(s)
            extracted.extend(temp_str2)
        # print(extracted)
    dict_ct[nct_no] = extracted
    # print()
print(dict_ct)

{'NCT00579683': [], 'NCT00671671': ['Small', 'Molecule', 'Agent', 'PF-868554'], 'NCT02179671': ['Gefitinib', 'AZD9291', 'Selumetinib', 'Docetaxel', 'Tremelimumab'], 'NCT02259114': ['Birabresib'], 'NCT02395601': ['CPI-1205'], 'NCT01949883': ['CPI-0610'], 'NCT02157636': ['CPI-0610'], 'NCT04685811': ['68Ga-PSMA-cyclotron', 'Versus', '68Ga-PSMA-generator'], 'NCT02038673': ['ASP5878'], 'NCT03157310': ['Gefitinib'], 'NCT02444884': ['MLN8237'], 'NCT03722576': ['Vidofludimus', 'calcium'], 'NCT02916537': ['CTT1057'], 'NCT04313166': ['CuIIATSM'], 'NCT01533181': ['Linsitinib', 'Topotecan', 'Hydrochloride'], 'NCT01605526': ['RO5045337', 'doxorubicin'], 'NCT02419417': ['BMS-986158'], 'NCT01462175': ['RO5503781'], 'NCT01334177': ['TLR8', 'agonist', 'VTX-2337'], 'NCT02777710': ['Pexidartinib', 'Durvalumab'], 'NCT00939770': ['Crizotinib'], 'NCT02806817': ['ME-344', 'Bevacizumab'], 'NCT01261754': [], 'NCT00712829': ['123-I-MIP-1072', '123-I-MIP-1095'], 'NCT03626415': ['PF-04965842'], 'NCT00835419': ['P

In [122]:
dict_ct

{'NCT00000180': ['AIT-082'],
 'NCT00055380': [],
 'NCT00116376': ['AEE788'],
 'NCT00118456': ['AEE788'],
 'NCT00124280': ['RAD001'],
 'NCT00158782': ['GW786034', 'lapatinib'],
 'NCT00205257': [],
 'NCT00257114': ['bortezomib'],
 'NCT00259753': ['Bevasiranib'],
 'NCT00306904': ['bevasiranib'],
 'NCT00312780': ['XL784'],
 'NCT00364780': ['XL647'],
 'NCT00367471': ['lapatinib', 'carboplatin', 'trastuzumab', 'paclitaxel'],
 'NCT00370383': ['Erlotinib', 'Satraplatin'],
 'NCT00390468': ['Tandutinib'],
 'NCT00455039': ['GW572016'],
 'NCT00490139': ['Lapatinib'],
 'NCT00540176': ['Dichloroacetate', 'DCA'],
 'NCT00541190': [],
 'NCT00543049': ['Sunitinib', 'SUNITINIB'],
 'NCT00550485': ['escitalopram'],
 'NCT00553358': ['Lapatinib', 'Paclitaxel'],
 'NCT00566410': ['Dichloroacetate'],
 'NCT00567554': ['epirubicin',
  'cyclophosphamide',
  'docetaxel',
  'epirubicin',
  'cyclophosphamide',
  'docetaxel',
  'bevacizumab',
  'paclitaxel',
  'paclitaxel',
  'everolimus',
  'RAD001',
  'epirubicin',


## drugbank vocabulary

In [143]:
df_drugbank = pd.read_csv('/content/drive/MyDrive/pharmaHacks/drugbank_vocabulary.csv', index_col=0)
df_drugbank['syno'] = [str(x).lower() for x in df_drugbank['Synonyms']]
df_drugbank['common_name'] = [str(x).lower() for x in df_drugbank['Common name']]
df_drugbank['synonyms'] = df_drugbank[['common_name', 'syno']].agg(' | '.join, axis=1)
df_drugbank = df_drugbank.drop(columns=['syno', 'common_name'])
df_drugbank

,Accession Numbers,Common name,CAS,UNII,Synonyms,Standard InChI Key,synonyms
DrugBank ID,,,,,,,
DB00001,BIOD00024 | BTD00024,Lepirudin,138068-37-8,Y43GF64R34,Hirudin variant-1 | Lepirudin | Lepirudin reco...,NaN,lepirudin | hirudin variant-1 | lepirudin | le...
DB00002,BIOD00071 | BTD00071,Cetuximab,205923-56-4,PQX0D8J21J,Cetuximab | Cétuximab | Cetuximabum,NaN,cetuximab | cetuximab | cétuximab | cetuximabum
DB00003,BIOD00001 | BTD00001,Dornase alfa,143831-71-4,953A26OA1Y,Deoxyribonuclease (human clone 18-1 protein mo...,NaN,dornase alfa | deoxyribonuclease (human clone ...
DB00004,BIOD00084 | BTD00084,Denileukin diftitox,173146-27-5,25E79B5CTM,Denileukin | Denileukin diftitox | Interleukin...,NaN,denileukin diftitox | denileukin | denileukin ...
DB00005,BIOD00052 | BTD00052,Etanercept,185243-69-0,OP401G7OJC,Etanercept | etanercept-szzs | etanercept-ykro...,NaN,etanercept | etanercept | etanercept-szzs | et...
...,...,...,...,...,...,...,...
DB16742,NaN,RP-67580,135911-02-3,49U9M41BGY,NaN,NaN,rp-67580 | nan
DB16743,NaN,Nolpitantium chloride,153050-21-6,22O6XI63E0,NaN,NaN,nolpitantium chloride | nan
DB16744,NaN,CP-96345,132746-60-2,W22ILA2I52,NaN,NaN,cp-96345 | nan


In [144]:
tt = np.where([(not math.isnan(test)) and test != -1.0 for test in df_drugbank['synonyms'].str.find('ME-344')])[0]
df_drugbank.iloc[tt,:]

,Accession Numbers,Common name,CAS,UNII,Synonyms,Standard InChI Key,synonyms
DrugBank ID,,,,,,,


In [159]:
dict_drugbank = {}
all_dbid = set()
for key in dict_ct.keys():
    db_ids = []
    for value in dict_ct.get(key):
        # print(f'value: {value}')
        matches = np.where([(not math.isnan(test)) and test != -1.0 for test in df_drugbank['synonyms'].str.find(value.lower())])[0]
        li = list(df_drugbank.index[matches])
        db_ids.extend(li)
        all_dbid |= set(li)
    # print(db_ids)
    dict_drugbank[key] = db_ids
print(dict_drugbank)
print(all_dbid)

{'NCT00579683': [], 'NCT00671671': ['DB14266', 'DB14305', 'DB14358', 'DB14360', 'DB14711', 'DB15497', 'DB05643', 'DB13881'], 'NCT02179671': ['DB00317', 'DB11689', 'DB01248', 'DB11771'], 'NCT02259114': ['DB15189'], 'NCT02395601': ['DB14581'], 'NCT01949883': [], 'NCT02157636': [], 'NCT04685811': [], 'NCT02038673': [], 'NCT03157310': ['DB00317'], 'NCT02444884': [], 'NCT03722576': ['DB15446', 'DB00258', 'DB00326', 'DB01164', 'DB01373', 'DB06724', 'DB09116', 'DB10516', 'DB11093', 'DB11126', 'DB11168', 'DB11211', 'DB11264', 'DB11348', 'DB13142', 'DB13231', 'DB13365', 'DB13372', 'DB13590', 'DB13612', 'DB13800', 'DB13829', 'DB13962', 'DB14481', 'DB14483', 'DB14497', 'DB14577', 'DB14598', 'DB14684', 'DB15533', 'DB15648', 'DB15678', 'DB15816', 'DB15912', 'DB15962'], 'NCT02916537': [], 'NCT04313166': [], 'NCT01533181': ['DB06075', 'DB01030', 'DB00387', 'DB06016', 'DB06261', 'DB09069', 'DB16619'], 'NCT01605526': ['DB00997', 'DB05706', 'DB06013', 'DB06420', 'DB11616', 'DB12755'], 'NCT02419417': ['D

In [160]:
json.dump(dict_drugbank, open("output_txt/dict_drugbank.txt",'w'))
with open('output_txt/all_dbid.txt', 'w') as f:
    for item in all_dbid:
        f.write("%s\n" % item)
# dict_drugbank = json.load(open("dict_drugbank.txt"))

## drugbank id -> SMILES

In [162]:
df_smiles = pd.read_csv('drugbank_smiles.csv')
df_smiles

,kind,value,source,parent_key
0,SMILES,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...,ChemAxon,DB00006
1,SMILES,CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(N)=N)NC(=...,ChemAxon,DB00007
2,SMILES,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...,ChemAxon,DB00014
3,SMILES,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...,ChemAxon,DB00027
4,SMILES,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...,ChemAxon,DB00035
...,...,...,...,...
11292,SMILES,NCC1=CC(=NC(OC2=CC(=CC=C2)C(=O)N2C[C@@H](O)[C@...,ChemAxon,DB16735
11293,SMILES,CC(=O)NCC(=O)N[C@@H](CCC(N)=O)C(=O)N[C@@H](CC1...,ChemAxon,DB16737
11294,SMILES,CC(C)C1=CC=C2N(CC3=CC=C(Cl)C=C3)C(CC(C)(C)C(O)...,ChemAxon,DB16739
11295,SMILES,[H][C@@]1(OB(O[C@]1([H])[C@H](O)CO)[C@H](CC(C)...,ChemAxon,DB16741


In [199]:
df_smiles.shape, df_smiles[df_smiles['parent_key'].isin(all_dbid)].shape

((11297, 4), (7473, 4))

In [202]:
# df_conversion = pd.DataFrame(data=None)
# for key in dict_drugbank.keys():
#     for value in dict_drugbank.get(key):
#         df_temp = {'NCT Number': key, 'DrugBank ID': value}
#         df_conversion = df_conversion.append(df_temp, ignore_index = True)
# df_conversion

,NCT Number,DrugBank ID
0,NCT00671671,DB14266
1,NCT00671671,DB14305
2,NCT00671671,DB14358
3,NCT00671671,DB14360
4,NCT00671671,DB14711
...,...,...
26602,NCT00306904,DB06642
26603,NCT01638546,DB00853
26604,NCT01638546,DB07232
26605,NCT00935792,DB00087


In [218]:
df_conversion_smiles = pd.DataFrame()
for key in dict_drugbank.keys():
    for value in dict_drugbank.get(key):
        smiles = np.array(df_smiles['value'])[np.where(df_smiles['parent_key'] == value)[0]]
        if smiles.shape[0]>0:
            # print(smiles[0])
            df_temp = {'NCT Number': key, 'DrugBank ID': value, 'SMILES': smiles[0]}
            df_conversion_smiles = df_conversion_smiles.append(df_temp, ignore_index = True)
df_conversion_smiles

,NCT Number,DrugBank ID,SMILES
0,NCT02179671,DB00317,COC1=C(OCCCN2CCOCC2)C=C2C(NC3=CC(Cl)=C(F)C=C3)...
1,NCT02179671,DB11689,CN1C=NC2=C(F)C(NC3=C(Cl)C=C(Br)C=C3)=C(C=C12)C...
2,NCT02179671,DB01248,[H][C@@]1(C[C@@]2(O)[C@@H](OC(=O)C3=CC=CC=C3)[...
3,NCT02259114,DB15189,CC1=C(C)C2=C(S1)N1C(C)=NN=C1[C@H](CC(=O)NC1=CC...
4,NCT02395601,DB14581,COC1=C(CNC(=O)C2=C(C)N([C@H](C)C3CCN(CC(F)(F)F...
...,...,...,...
22507,NCT01453153,DB15057,C[C@H](NP(=O)(OC[C@H]1O[C@@H](N2C=CC(N)=NC2=O)...
22508,NCT01729728,DB06204,CC[C@H]([C@@H](C)CN(C)C)C1=CC(O)=CC=C1
22509,NCT01638546,DB00853,CN1N=NC2=C(N=CN2C1=O)C(N)=O
22510,NCT01638546,DB07232,C[C@@]1(CCCN1)C1=NC2=CC=CC(C(N)=O)=C2N1


In [219]:
df_conversion_smiles.to_csv('output_txt/df_conversion.csv')